In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HK2']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [8]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [9]:
list(zip(longforms, counts))

[('hexokinase 2', 397),
 ('hexokinase', 12),
 ('hexokinase2', 7),
 ('human kidney 2', 6),
 ('human kidney', 5),
 ('hexokinase isoform 2', 3),
 ('human kidney proximal tubular cells', 3),
 ('hexokinase 2 isoform', 2),
 ('hk1 and 2', 2),
 ('human kidney epithelial', 2),
 ('human kidney proximal tubular', 2)]

In [10]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [11]:
names

{'HGNC:4923': 'HK2',
 'MESH:D007668': 'Kidney',
 'MESH:D002477': 'Cells',
 'HGNC:6318': 'KIF2A'}

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 02:30:00] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [13]:
result = [grounding_map, names, pos_labels]

In [14]:
result

[{'hexokinase': 'HGNC:4923',
  'hexokinase 2': 'HGNC:4923',
  'hexokinase 2 isoform': 'HGNC:4923',
  'hexokinase isoform 2': 'HGNC:4923',
  'hexokinase2': 'HGNC:4923',
  'hk1 and 2': 'ungrounded',
  'human kidney': 'ungrounded',
  'human kidney 2': 'ungrounded',
  'human kidney epithelial': 'ungrounded',
  'human kidney proximal tubular': 'ungrounded',
  'human kidney proximal tubular cells': 'ungrounded'},
 {'HGNC:4923': 'HK2'},
 ['HGNC:4923']]

In [15]:
grounding_map, names, pos_labels = [{'hexokinase': 'HGNC:4923',
  'hexokinase 2': 'HGNC:4923',
  'hexokinase 2 isoform': 'HGNC:4923',
  'hexokinase isoform 2': 'HGNC:4923',
  'hexokinase2': 'HGNC:4923',
  'hk1 and 2': 'ungrounded',
  'human kidney': 'ungrounded',
  'human kidney 2': 'ungrounded',
  'human kidney epithelial': 'ungrounded',
  'human kidney proximal tubular': 'ungrounded',
  'human kidney proximal tubular cells': 'ungrounded'},
 {'HGNC:4923': 'HK2'},
 ['HGNC:4923']]

In [16]:
excluded_longforms = ['hk1 and 2']

In [25]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [26]:
additional_entities = {'HGNC:6318': ['KIF2A', ['HK2']]}

In [40]:
unambiguous_agent_texts = {'HGNC:6318': ['KIF2A', ['KIF2A']]}

In [41]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [42]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [43]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [44]:
intersection1

[('HGNC:6318', 'HGNC:6318', 0)]

In [45]:
intersection2

[('HGNC:4923', 'HGNC:6318', 0), ('ungrounded', 'HGNC:6318', 0)]

In [46]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [47]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [48]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 02:37:19] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 02:37:25] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9554565041383519 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [49]:
classifier.stats

{'label_distribution': {'HGNC:4923': 324, 'ungrounded': 14, 'HGNC:6318': 54},
 'f1': {'mean': 0.955457, 'std': 0.023451},
 'precision': {'mean': 0.948094, 'std': 0.013342},
 'recall': {'mean': 0.971018, 'std': 0.025492},
 'HGNC:4923': {'f1': {'mean': 0.968686, 'std': 0.00838},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.939401, 'std': 0.015718}},
 'ungrounded': {'f1': {'mean': 0.393333, 'std': 0.33493},
  'pr': {'mean': 0.3, 'std': 0.266667},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'HGNC:6318': {'f1': {'mean': 0.877684, 'std': 0.114558},
  'pr': {'mean': 0.8, 'std': 0.176279},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [50]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [51]:
disamb.dump(model_name, results_path)

In [52]:
print(disamb.info())

Disambiguation model for HK2

Produces the disambiguations:
	HK2*	HGNC:4923
	KIF2A*	HGNC:6318

Class level metrics:
--------------------
Grounding	Count	F1     
  HK2*	324	0.96869
KIF2A*	 54	0.87768
Ungrounded	 14	0.39333

Weighted Metrics:
-----------------
	F1 score:	0.95546
	Precision:	0.94809
	Recall:		0.97102

* Positive labels
See Docstring for explanation



In [61]:
model_to_s3(disamb)

In [58]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [59]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:6318']

In [60]:
len(texts)

0